In [ ]:
import pandas as pd
import ast
import gdown
########## 1. TRANSFORMACIONES #############
'''1.0 Crear acceso a dataframe a partir de las bases de datos:movies_dataset.csv y credits.csv '''
url_movies = 'https://drive.google.com/u/0/uc?id=1Rp7SNuoRnmdoQMa5LWXuK4i7W1ILblYb&export=download'
url_credits = 'https://drive.google.com/uc?id=1lMGJUWVVVRPO00ZWqzEJZIxFhSqtfmAB&export=download'

csv_movies = 'movies_dataset_orig.csv'
csv_credits = 'credits_orig.csv'

gdown.download(url_movies, csv_movies, quiet=False, format='csv')
gdown.download(url_credits, csv_credits, quiet=False, format='csv')

df = pd.read_csv(csv_movies, low_memory=False)
df_credits = pd.read_csv(csv_credits, low_memory=False)

# -----------
''' 1.1 Acceso a los diccionarios sin desanidar:
    Convierte los valores de la columna "belongs_to_collection" (movies dataset) a diccionarios, asigna diccionario 
    vacío donde hay "nan"'''
columna_dict ='belongs_to_collection'
key1_dict = 'name'
df[columna_dict] = df[columna_dict].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})
'''Para acceder a los valores de esta columna, se pueden leer directamente los diccionarios, pero estos pueden tener 
   distintas estructuras y/o contener o no la key "name", entonces hay que agregar condicionantes (if isinstance..etc)'''
key1_dict_values = df[columna_dict].apply(lambda x: x[key1_dict] if isinstance(x, dict) and key1_dict in x else None)
'''Para hacer legibles las LISTAS de DICCIONARIOS:
   Convierte las listas a diccionarios para consultas posteriores'''
columna_list_variables_movies= ['genres','production_companies','production_countries', 'spoken_languages']
'''Del dataset movies'''
for columna_list in columna_list_variables_movies:
    df[columna_list] = df[columna_list].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
'''Del dataset credits'''
columna_list_variables_credits= ['cast','crew']
for columna_list in columna_list_variables_credits:
    df_credits[columna_list] = df_credits[columna_list].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

# -----------
''' 1.2 y 1.3. Manejo de valores nulos:
    Reemplaza los valores nulos por el valor 0'''
df['revenue'] = df['revenue'].fillna(0) 
df = df.dropna(subset=['release_date'])  # Elimina las filas del dataframe para las que no hay fecha de lanzamiento 
''' El campo budget tiene registros que no son números, p.e. "jsfhaksh.jpg", por ello se usa el parámetro errors = 'coerce' 
    el cual escribe NaN en los renglones que no contienen numeros y que por lo tanto no se pueden convertir a float, 
    luego reemplaza los NaN con 0'''
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['budget'] = df['budget'].fillna(0) 

# -----------
''' 1.4. Manejo de FECHAS
    Campos con fechas: release_date'''
df['release_date'] = pd.to_datetime(df['release_date'], errors = 'coerce')
df['release_year'] = df['release_date'].dt.strftime('%Y')

# -----------
'''1.5. Calcular retorno de inversión
    Crea un nuevo dataframe para aplicar la operación requerida, recorriendo cada renglón y aplicando un criterio para 
    el caso de que budget o revenue sean 0 o Nan.
    Nuevo DF con las variables revenue y budget: '''
rb = pd.DataFrame({'REVENUE': df['revenue'], 'BUDGET': df['budget']})
''' Calcula el retorno. La palabra "return" esta reservada, por eso la variable se llamará "return_rate", aunque el titulo de 
    la columna si sea "return"'''
return_rate = rb.apply(lambda row: row['REVENUE'] / row['BUDGET']
                        if (row['BUDGET'] != 0) & (row['REVENUE'] != 0) else 0, axis=1)
''' Agrega la columna "return" al dataframe y lo rellena con los datos de return_rate'''
df['return'] = return_rate

# -----------
''' 1.6. Elimina columnas que no se van a necesitar
    Lista con los nombres de las columnas que se van a eliminar'''
col_to_elimin = ['video','imdb_id', 'adult','original_title','poster_path','homepage']
'''     Elimina del dataframe las columnas especificadas'''
df = df.drop(col_to_elimin, axis='columns')

# -----------
'''1.7. EXTRA: Asignar tipos de datos correspondientes:'''
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')
df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce')
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce')
df['id'] = pd.to_numeric(df['id'], errors='coerce')

# -----------
''' 1.8. EXTRA: Hacer join o merge entre los dos datasets para hacer las consultas en la API.
    Se hace in inner join, ya que solo se van a ocupar los registros que tengan coincidencias en ambos datasets'''
completo = pd.merge(df, df_credits, on='id')
'''Respaldo de la base de datos completa (despues de relacionar movies_dataset.csv con credits.csv)'''
# file_path = r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\DATA\PI01HENRY_Dataset_RMCE.csv'
# completo.to_csv(file_path, index=False)

# -----------
''' 1.9. EXTRA: Preparar y exportar los archivos que se van a necesitar en la API'''
'''Dataset MOVIES FILTRADO. Crea una lista con los nombres de las columnas que se van a eliminar y luego las elimina con drop. Exporta el
dataframe como un archivo csv que se guarda en el workspace'''
elim_col_df = ['belongs_to_collection','budget','genres','id','original_language','overview','production_companies',
               'production_countries','revenue','runtime','spoken_languages','status','tagline','return']
movies = df.drop(elim_col_df, axis=1)
movies.to_csv('movies_dataset_filtrado_RMCE.csv', index=False)

'''Dataset CAST. Crea una lista con los nombres de las columnas que se van a eliminar y luego las elimina con drop. Exporta el
dataframe como un archivo csv que se guarda en el workspace'''
elim_col_actor = ['belongs_to_collection','budget','genres','id','original_language','overview','popularity','production_companies',
                     'production_countries', 'release_date','revenue','runtime','spoken_languages','status','tagline','vote_average',
                     'vote_count','release_year','crew']
movies_cast_actor = completo.drop(elim_col_actor,axis=1)
movies_cast_actor.to_csv('movies_cast_actor_RMCE.csv',index=False)

'''Dataset CREW. Crea una lista con los nombres de las columnas que se van a eliminar y luego las elimina con drop. Exporta el
dataframe como un archivo csv que se guarda en el workspace'''
elim_col_dir = ['belongs_to_collection','genres','id','original_language','overview','popularity','production_companies',
                     'production_countries','runtime','spoken_languages','status','tagline','vote_average',
                     'vote_count','release_year','cast']

movies_crew_dir = completo.drop(elim_col_dir,axis=1)
movies_crew_dir.to_csv('movies_crew_director_RMCE.csv',index=False)